## Youtube API 활용

### 1. 검색 리스트 조회

In [ ]:
# Step 1: 필요한 라이브러리 임포트
import requests
import pandas as pd
import json
import datetime
# Step 2: API 키 설정
API_KEY = "YOUR_API_KEY"


#%% Step 3: 검색 기능 구현
# YouTube Data API 검색 엔드포인트
search_url = "https://www.googleapis.com/youtube/v3/search"

search_term = '아이유'
# 검색어와 검색 조건 설정
search_params = {
    'key': API_KEY,
    'q': search_term,  # 예: 'BTS'
    'part': 'snippet',
    'maxResults': 5,  # 검색 결과 수 (최대 50)
    'type': 'video'   # 동영상만 검색
}

# API 호출
response = requests.get(search_url, params=search_params)

# 결과 확인
if response.status_code == 200:
    search_data = response.json()
    print("검색 성공!")
    print(f"검색된 동영상 수: {len(search_data['items'])}")
else:
    print(f"검색 실패! 에러 코드: {response.status_code}")
    print(response.text)

#### 검색 결과를 데이터프레임으로 변환

In [ ]:

# 검색 결과를 저장할 리스트
video_list = []

# 검색 결과 처리
for item in search_data['items']:
    video = {
        'video_id': item['id']['videoId'],
        'title': item['snippet']['title'],
        'channel_title': item['snippet']['channelTitle'],
        'published_at': item['snippet']['publishedAt'],
        'description': item['snippet']['description']
    }
    video_list.append(video)

# 데이터프레임 생성
search_df = pd.DataFrame(video_list)
print("\n검색 결과 데이터:")
print(search_df)

### 2. 영상 상세정보 조회
- 조회수, 좋아요 수, 코멘트 수 

In [ ]:
#%% Step 5: 동영상 상세 정보 가져오기
# 동영상 상세정보 API 엔드포인트
video_url = "https://www.googleapis.com/youtube/v3/videos"

# 상세정보를 저장할 리스트
stats_list = []

# 각 동영상의 상세 정보 수집
for video_id in search_df['video_id']:
    # API 매개변수 설정
    video_params = {
        'key': API_KEY,
        'id': video_id,
        'part': 'statistics'
    }
    
    # API 호출
    video_response = requests.get(video_url, params=video_params)
    
    if video_response.status_code == 200:
        video_data = video_response.json()
        if video_data['items']:
            stats = video_data['items'][0]['statistics']
            stats_list.append({
                'video_id': video_id,
                'view_count': int(stats.get('viewCount', 0)),
                'like_count': int(stats.get('likeCount', 0)),
                'comment_count': int(stats.get('commentCount', 0))
            })
            print(f"ID: {video_id} 상세정보 수집 완료")

# 상세정보 데이터프레임 생성
stats_df = pd.DataFrame(stats_list)
print("\n상세정보 데이터:")
print(stats_df)

### 3.두 데이터 병합 및 저장

In [ ]:
#%% Step 6: 두 데이터프레임 병합
# video_id를 기준으로 두 데이터프레임 병합
final_df = pd.merge(search_df, stats_df, on='video_id')

# URL 컬럼 추가
final_df['video_url'] = 'https://www.youtube.com/watch?v=' + final_df['video_id']

print("\n최종 데이터:")
print(final_df)

#%% Step 7: 데이터 저장
# 현재 시간을 파일명에 포함
json_filename = f'youtube_results_{search_term}_{len(final_df)}개.json'

# DataFrame을 JSON으로 변환하여 저장
# orient='records'는 데이터를 리스트 형태의 JSON으로 저장
# force_ascii=False는 한글 등의 유니코드 문자를 그대로 저장
# indent=2는 JSON 파일을 보기 좋게 들여쓰기
final_df.to_json(json_filename, orient='records', force_ascii=False, indent=2)

#### 4.데이터 분석

In [ ]:
# 조회수 기준 상위 3개 동영상
print("\n조회수 TOP 3:")
print(final_df.nlargest(3, 'view_count')[['title', 'view_count', 'video_url']])

# 좋아요 수 기준 상위 3개 동영상
print("\n좋아요 TOP 3:")
print(final_df.nlargest(3, 'like_count')[['title', 'like_count', 'video_url']])

# 기본 통계 정보
print("\n통계 정보:")
print(final_df[['view_count', 'like_count', 'comment_count']].describe())